# Sequence to Sequence Learning with Neural Networks
- reference code: https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### tokenizer를 위한 spacy 다운 (영어: en / de: 독일어)

In [3]:
!python -m spacy download en
!python -m spacy download de

     |████████████████████████████████| 12.0 MB 9.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/en_core_web_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 9.3 MB/s eta 0:00:01    |█████▌                          | 2.6 MB 9.3 MB/s eta 0:00:02     |█████████████▍                  | 6.3 MB 9.3 MB/s eta 0:00:01     |█████████████████████████       | 11.7 MB 9.3 MB/s eta 0:00:01
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-py3-none-any.whl size=14907581 sha256=632b74702c9e9a6765b2324953d6369c359a3636780a5cbf757dab4c054b0ee3
  Stored in directory: /tmp/pip-ephem-wheel-cache-8anvcvzz/wheels/75/30/c3/ea1c6002eede7f49c8ab017ce62a2981a87b1cd39fab6e6a65
Successfully built de-core-news-sm


✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/de_core_news_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

In [5]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

### example for tokenizer
-> reverse!

In [6]:
tokenize_en('Hello. My name is Karlie.')

['.', 'Karlie', 'is', 'name', 'My', '.', 'Hello']

In [7]:
SRC = Field(tokenize = tokenize_de,
           init_token = '<sos>',
           eos_token = '<eos>',
            unk_token = '<unk>',
           lower = True)

TRG = Field(tokenize = tokenize_de,
           init_token = '<sos>',
           eos_token = '<eos>',
             unk_token = '<unk>',
           lower = True)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Download the data (Multi30k)

In [8]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                   fields = (SRC, TRG))
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 949kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 168kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 159kB/s]
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [9]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']}


In [10]:
SRC.build_vocab(train_data, max_size = 16000)
TRG.build_vocab(train_data, max_size = 8000)

In [67]:
# vocab lists
[val for val in SRC.vocab.stoi.items()][:10]

[('<unk>', 0),
 ('<pad>', 1),
 ('<sos>', 2),
 ('<eos>', 3),
 ('.', 4),
 ('ein', 5),
 ('einem', 6),
 ('in', 7),
 ('eine', 8),
 (',', 9)]

In [12]:
print(f"src vocab size: {len(SRC.vocab)}")
print(f"trg vocab size: {len(TRG.vocab)}")

src vocab size: 16004
trg vocab size: 8004


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device,
    sort_key=lambda x:len(x.src), # src 문장을 길이로 sort,
    sort_within_batch = True # 이걸 하는게 의미가 있나요?
)

/opt/conda/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [15]:
print(f"number of train batch: {len(train_iterator)}")
print(f"number of valid batch: {len(valid_iterator)}")
print(f"number of test batch: {len(test_iterator)}")

number of train batch: 227
number of valid batch: 8
number of test batch: 8


In [16]:
## data check
cnt = 0
for a , b in train_iterator:
    print('source text')
    print(a[0].shape)
    print(a[0])
#     print(a.size())
    print()
    print('target text')
    print(a[1].shape)
    print(a[1])
    cnt += 1
    print('********************')
    if cnt == 2:
        break

/opt/conda/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


source text
torch.Size([12, 128])
tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  4,   4,   4,  ...,   4,   4,   4],
        [155, 969, 348,  ...,  21, 362,  34],
        ...,
        [ 45, 608,  36,  ...,  41,  16, 116],
        [ 18,  13,   8,  ...,  18,   8,   5],
        [  3,   3,   3,  ...,   3,   3,   3]], device='cuda:0')

target text
torch.Size([20, 128])
tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  5,   5,   5,  ...,   5,   5,   5],
        [114, 633,  47,  ..., 297,  76,  39],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
********************
source text
torch.Size([12, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,    4,  ...,    4,    4,    4],
        [ 186, 4709, 5785,  ...,  979,  248,  137],
        ...,
        [  13,   16,  167,  ...,   16,   16, 6599],
        [   5,    8, 1016,  ...,    8,   

## model

### encoder
- return: 
1. outputs: the top-layer hidden state for each time-step
2. hidden: the final hidden state for each layer, $h_T$, stacked on top of each other
3. cell: the final cell state for each layer, $c_T$, stacked on top of each other

In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
#         print('@@@ encoder')
        
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
#         print('hidden', hidden.shape)
#         print('cell', cell.shape)
        return outputs, hidden, cell

### encoder output size check
hidden torch.Size([2, 128, 512])
cell torch.Size([2, 128, 512])

### decoder

encoder랑 구조는 같은데, input이 length 1로 들어가게 하는 거랑 prediction값을 dim = 1로 축소하는 거만 다름

In [18]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size] # 1: 문장 길이
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

## Seq2Seq
For the final part of the implemenetation, we'll implement the seq2seq model. This will handle:

- receiving the input/source sentence
- the encoder to produce the context vectors
- the decoder to produce the predicted output/target sentence

### teacher forcing

In [31]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    #src, trg, teacher_forcing_ratio = 0.5):
    def forward(self,src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden, cell = self.encoder(src)
        

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs
            

In [53]:
model = Seq2Seq(enc, dec, device).to(device)

In [54]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(16004, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(8004, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=8004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [55]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,608,516 trainable parameters


In [56]:
optimizer = optim.SGD(model.parameters(), lr = 0.7, momentum=0)

In [57]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [58]:
def train(model, iterator, optimizer, criterion, clip,epoch):
    model.train()
    epoch_loss = 0
    
    iter_check = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
               
        # custom scheduler
        gamma = 0.5      
        if epoch + 1 > 5 and i % (len(iterator) // 2) == 0 and i != 0:
            for group in optimizer.param_groups:
                group['lr'] = group['lr'] * gamma
                iter_check += 1
        
                print(iter_check,group['lr'] )

     
        optimizer.step()
        epoch_loss += loss.item()
    print('Epoch-{0} lr: {1}'.format(epoch+ 1, optimizer.param_groups[0]['lr']))
    return epoch_loss / len(iterator),iter_check

In [59]:
def evaluate(model, iterator, criterion):    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [60]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, iter_check = train(model, train_iterator, optimizer, criterion, CLIP,epoch)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # epoch 7.5 에 save
    if epoch+1 == 7 and iter_check == 1:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch-1 lr: 0.7
Epoch: 01 | Time: 0m 32s
	Train Loss: 5.700 | Train PPL: 298.879
	 Val. Loss: 5.222 |  Val. PPL: 185.266
Epoch-2 lr: 0.7
Epoch: 02 | Time: 0m 32s
	Train Loss: 5.041 | Train PPL: 154.657
	 Val. Loss: 4.873 |  Val. PPL: 130.678
Epoch-3 lr: 0.7
Epoch: 03 | Time: 0m 32s
	Train Loss: 4.926 | Train PPL: 137.882
	 Val. Loss: 4.823 |  Val. PPL: 124.346
Epoch-4 lr: 0.7
Epoch: 04 | Time: 0m 32s
	Train Loss: 4.870 | Train PPL: 130.286
	 Val. Loss: 4.848 |  Val. PPL: 127.507
Epoch-5 lr: 0.7
Epoch: 05 | Time: 0m 32s
	Train Loss: 4.836 | Train PPL: 125.909
	 Val. Loss: 4.836 |  Val. PPL: 125.945
1 0.35
2 0.175
Epoch-6 lr: 0.175
Epoch: 06 | Time: 0m 32s
	Train Loss: 4.794 | Train PPL: 120.754
	 Val. Loss: 4.778 |  Val. PPL: 118.905
1 0.0875
2 0.04375
Epoch-7 lr: 0.04375
Epoch: 07 | Time: 0m 32s
	Train Loss: 4.757 | Train PPL: 116.434
	 Val. Loss: 4.765 |  Val. PPL: 117.322
1 0.021875
2 0.0109375
Epoch-8 lr: 0.0109375
Epoch: 08 | Time: 0m 32s
	Train Loss: 4.749 | Train PPL: 115.481
	 V

In [62]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.787 | Test PPL: 119.999 |
